<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/examples/TourismSmall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hierarchical Reconciliation for TourismSmall dataset

Large collections of time series organized into structures at different aggregation levels of-
ten require their forecasts to follow their aggregation constraints, which poses the challenge
of creating novel algorithms capable of coherent forecasts.

The HierarchicalForecast package provides a wide collection of Python implementations of hierarchical forecasting algorithms that follow classic hierarchical reconciliation.

In [ ]:
!pip install hierarchicalforecast
!pip install statsforecast datasetsforecast

In [ ]:
import numpy as np
import pandas as pd

#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData
#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut
# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import auto_arima, naive

In this example we will use the `TourismSmall` dataset. The following cell obtaines the time series for the different levels in the hierarchy, the summing matrix `S` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
Y_df, S, tags = HierarchicalData.load('./data', 'TourismSmall')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

We split the dataframe in train/test splits.

In [ ]:
Y_df_test = Y_df.groupby('unique_id').tail(12)
Y_df_train = Y_df.drop(Y_df_test.index)

In [ ]:
Y_df_test = Y_df_test.set_index('unique_id')
Y_df_train = Y_df_train.set_index('unique_id')

The following cell computes the *base forecast* for each time series using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecast but they are not coherent.

In [ ]:
fcst = StatsForecast(df=Y_df_train, models=[(auto_arima,12), naive], freq='M', n_jobs=-1)
Y_hat_df = fcst.forecast(h=12)

INFO:statsforecast.core:Computing forecasts
/Users/fedex/opt/miniconda3/envs/hierarchicalforecast/lib/python3.10/site-packages/statsforecast/arima.py:1041: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj['var_coef']))))
/Users/fedex/opt/miniconda3/envs/hierarchicalforecast/lib/python3.10/site-packages/statsforecast/arima.py:1041: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj['var_coef']))))
/Users/fedex/opt/miniconda3/envs/hierarchicalforecast/lib/python3.10/site-packages/statsforecast/arima.py:1041: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj['var_coef']))))
/Users/fedex/opt/miniconda3/envs/hierarchicalforecast/lib/python3.10/site-packages/statsforecast/arima.py:1041: RuntimeWarning: invalid value encountered in sqrt
  return any(np.isnan(np.sqrt(np.diag(obj['var_coef']))))
/Users/fedex/opt/miniconda3/envs/hierarchicalforecast/lib/python3.10/site-pa

The following cell makes the previous forecasts coherent. The used methods to make the forecasts coherent are:
- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [ ]:
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(level='Country/Purpose/State', top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df, Y_df_train, S, tags)

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluator.evaluate(Y_h=Y_rec_df, Y_test=Y_df_test, 
                   tags=tags, benchmark='naive')

,,auto_arima_season_length-12,naive,auto_arima_season_length-12/BottomUp,naive/BottomUp,auto_arima_season_length-12/TopDown_method-forecast_proportions,naive/TopDown_method-forecast_proportions,auto_arima_season_length-12/MiddleOut_level-Country/Purpose/State_top_down_method-forecast_proportions,naive/MiddleOut_level-Country/Purpose/State_top_down_method-forecast_proportions
level,metric,,,,,,,,
Overall,mse-scaled,0.958433,1.0,0.908254,1.0,0.973065,1.0,0.896154,1.0
Country,mse-scaled,1.05191,1.0,0.978696,1.0,1.05191,1.0,0.950772,1.0
Country/Purpose,mse-scaled,0.903941,1.0,0.861644,1.0,0.887653,1.0,0.830192,1.0
Country/Purpose/State,mse-scaled,0.909366,1.0,0.867018,1.0,0.973846,1.0,0.909366,1.0
Country/Purpose/State/CityNonCity,mse-scaled,0.84693,1.0,0.84693,1.0,0.949697,1.0,0.885367,1.0
